<a href="https://colab.research.google.com/github/shoshanabezalel/ClubMembershipApplication/blob/master/SmartNikud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U transformers
!pip install torch torchvision torchaudio
!pip install PyPDF2 pdfplumber python-docx
!pip install datasets evaluate
!pip install accelerate
!pip install PyMuPDF

# הפעלת GPU
import torch

import re
import PyPDF2
import pdfplumber
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
import unicodedata
from google.colab import files
import fitz
import sys
print(f"Python version: {sys.version}")

# בדיקת התקנה מוצלחת
import transformers

print("✅ כל החבילות הותקנו בהצלחה")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [4]:
test_text = """
Q0@79@.ב99@
38@
29@
@06לפני @16איזהו שלושה ימים אסור לשאת ולתת עמו
@23גכולהו @23מישום דאיל ומודה @23לעבודה זרת ביום אידו
"""

# טקסט מורכב יותר מהתלמוד
complex_test = """
@23במידי דלא הדר בעין, כגון בהמה וכלים @33דכתיב (@43שמות כב)
'אם כסף תלוה את עמי', דמלתא להוצאה נתינה, ומשלם לו מעות
אחרים: @13שמח הוא לאחר זמן. @23למחר:@03 גמ' @13יום אידם.
"""

# שמירה כקובץ לבדיקה
with open('test_talmud.txt', 'w', encoding='utf-8') as f:
    f.write(test_text)

print("✅ קובץ בדיקה נוצר")

✅ קובץ בדיקה נוצר


In [34]:

# בדיקה סופית מקיפה של כל המערכת
def final_system_test():
    """בדיקה סופית של כל רכיבי המערכת"""

    print("🏁 בדיקה סופית של המערכת")

    # בדיקה 1: זיהוי סימנים
    test_formatting = "Q0@79@.ב99@ @23דכתיב @16איזהו 100@"
    segments = final_processor.smart_text_segmentation(test_formatting)
    formatting_count = sum(1 for s in segments if s['type'] == 'formatting')
    hebrew_count = sum(1 for s in segments if s['type'] == 'hebrew')

    print(f"✅ זיהוי סימנים: {formatting_count} סימנים, {hebrew_count} מילים עבריות")

    # בדיקה 2: ניקוד מילים
    test_words = ['דכתיב', 'רבי', 'אמר', 'הלכה', 'משנה']
    nikud_success = 0
    for word in test_words:
        result = advanced_nikud_word(word, final_processor.talmud_nikud_dict)
        if result != word:
            nikud_success += 1

    nikud_rate = (nikud_success / len(test_words)) * 100
    print(f"✅ ניקוד מילים: {nikud_rate}% הצלחה")

    # בדיקה 3: עיבוד טקסט מלא
    final_processor.processed_words = 0
    final_processor.successful_words = 0
    ai_processor = AITalmudNikudProcessor()

    test_full = "רבי יהודה אמר @23דכתיב בתורה הלכה למשה מסיני"
    result = ai_processor.process_text_to_nikud(test_full)
    final_stats = final_processor.get_success_statistics()

    print(f"✅ עיבוד מלא: {final_stats['success_rate']} הצלחה")
    print(f"✅ דוגמה: {result[:50]}...")

    # הערכה כללית
    overall_score = (100 + nikud_rate + float(final_stats['success_rate'].replace('%', ''))) / 3
    print(f"\n🎯 ציון כללי: {overall_score:.1f}/100")

    if overall_score >= 85:
        print("🎉 המערכת מוכנה לשימוש מקצועי!")
    elif overall_score >= 75:
        print("✅ המערכת עובדת טוב, יש מקום לשיפורים קלים")
    else:
        print("⚠️ יש צורך בשיפורים נוספים")

    return overall_score

# הפעלת הבדיקה הסופית
final_score = final_system_test()


🏁 בדיקה סופית של המערכת
✅ זיהוי סימנים: 4 סימנים, 2 מילים עבריות


AttributeError: 'str' object has no attribute 'remove_existing_nikud'

In [33]:
def advanced_nikud_word(self, word):
    """ניקוד מתקדם עם טיפול בקידומות וכללי דקדוק"""

    def remove_existing_nikud(self, text):
        """הסרת ניקוד קיים מהטקסט"""
        nikud_chars = '\u05B0\u05B1\u05B2\u05B3\u05B4\u05B5\u05B6\u05B7\u05B8\u05B9\u05BA\u05BB\u05BC\u05BD\u05BE\u05BF\u05C0\u05C1\u05C2\u05C3\u05C4\u05C5\u05C6\u05C7'
        return ''.join(char for char in text if char not in nikud_chars)

    # הסרת ניקוד קיים אם יש
    clean_word = self.remove_existing_nikud(word)

    # בדיקה ישירה במילון הבסיסי
    if clean_word in self.talmud_nikud_dict:
        return self.talmud_nikud_dict[clean_word]

    # טיפול בקידומות נפוצות

    # ו' החיבור
    if clean_word.startswith('ו') and len(clean_word) > 1:
        base_word = clean_word[1:]
        if base_word in self.talmud_nikud_dict:
            base_nikud = self.talmud_nikud_dict[base_word]
            # ו' החיבור לפני בגדכפת = וּ, אחרת = וְ
            if base_word[0] in 'בגדכפת':
                return 'וּ' + base_nikud
            else:
                return 'וְ' + base_nikud

    # ה' הידיעה
    if clean_word.startswith('ה') and len(clean_word) > 1:
        base_word = clean_word[1:]
        if base_word in self.talmud_nikud_dict:
            base_nikud = self.talmud_nikud_dict[base_word]
            # ה' הידיעה = הַ ברוב המקרים
            return 'הַ' + base_nikud

    # קידומות אחרות: ל, ב, כ, מ
    prefix_nikud_map = {
        'ל': 'לְ',  # ל' התכלית
        'ב': 'בְּ',  # ב' המקום
        'כ': 'כְּ',  # כ' הדמיון
        'מ': 'מִ'   # מ' המקום
    }

    for prefix_char, prefix_nikud in prefix_nikud_map.items():
        if clean_word.startswith(prefix_char) and len(clean_word) > 1:
            base_word = clean_word[1:]
            if base_word in self.talmud_nikud_dict:
                base_nikud = self.talmud_nikud_dict[base_word]
                return prefix_nikud + base_nikud

    # טיפול במקרים מיוחדים של תלמוד

    # ביטויים ארמיים נפוצים
    aramaic_patterns = {
        'דאמר': 'דַּאֲמַר',
        'דתניא': 'דְּתַנְיָא',
        'דהיינו': 'דְּהַיְנוּ',
        'משום': 'מִשּׁוּם',
        'בהדיא': 'בְּהַדְיָא'
    }

    if clean_word in aramaic_patterns:
        return aramaic_patterns[clean_word]

    # אם לא נמצא כלום - החזר את המילה המקורית
    return word

In [31]:
class FinalTalmudProcessor(TalmudNikudProcessor):
    """מעבד סופי לעיבוד קבצי PDF של התלמוד"""

    # מילון ניקוד מקיף לתלמוד בבלי
    talmud_nikud_dict = {
        # מילים בסיסיות
        'שלושה': 'שְׁלוֹשָׁה',
        'ימים': 'יָמִים',
        'אסור': 'אָסוּר',
        'לשאת': 'לָשֵׂאת',
        'ולתת': 'וְלָתֵת',
        'עמו': 'עִמּוֹ',

        # ביטויים תלמודיים
        'דכתיב': 'דִּכְתִיב',
        'דאמר': 'דַּאֲמַר',
        'מישום': 'מִשּׁוּם',
        'דאיל': 'דְּאִיל',
        'ומודה': 'וּמוֹדֶה',
        'בעלמא': 'בְּעָלְמָא',
        'דהיינו': 'דְּהַיְנוּ',

        # שמות תנאים ואמוראים
        'רבי': 'רַבִּי',
        'אמר': 'אָמַר',
        'יהודה': 'יְהוּדָה',
        'שמעון': 'שִׁמְעוֹן',
        'אליעזר': 'אֱלִיעֶזֶר',

        # מונחי הלכה
        'עבודה': 'עֲבוֹדָה',
        'זרה': 'זָרָה',
        'כוכבים': 'כּוֹכָבִים',
        'ומזלות': 'וּמַזָּלוֹת',

        # זמנים
        'ביום': 'בְּיוֹם',
        'אידו': 'אִידוּ',
        'לפני': 'לִפְנֵי',
        'אחרי': 'אַחֲרֵי'
    }

    def remove_existing_nikud(self, text):
        """הסרת ניקוד קיים מהטקסט"""
        nikud_chars = '\u05B0\u05B1\u05B2\u05B3\u05B4\u05B5\u05B6\u05B7\u05B8\u05B9\u05BA\u05BB\u05BC\u05BD\u05BE\u05BF\u05C0\u05C1\u05C2\u05C3\u05C4\u05C5\u05C6\u05C7'
        return ''.join(char for char in text if char not in nikud_chars)


    print(f"נוצר מילון עם {len(talmud_nikud_dict)} מילים")
    def __init__(self):
        super().__init__()
        self.version = "1.0"
        self.accuracy_target = 90.0

    def process_pdf_to_nikud(self, pdf_path):
        """עיבוד PDF מלא לניקוד עם שימור מבנה"""
        print("🎯 מתחיל עיבוד PDF מלא...")

        # חילוץ טקסט
        raw_text = self.extract_pdf_with_structure(pdf_path)
        if not raw_text:
            print("❌ לא הצלחתי לחלץ טקסט מה-PDF")
            return None

        print(f"📝 חולץ {len(raw_text)} תווים")

        # עיבוד הטקסט לניקוד
        self.ai_processor = AITalmudNikudProcessor()
        return self.ai_processor.process_text_to_nikud(raw_text)


  # יצירת המעבד הסופי
final_processor = FinalTalmudProcessor()

נוצר מילון עם 26 מילים
🚀 מפעיל מעבד PDF חכם לתלמוד (גרסה מורחבת)...


In [16]:
# מעבד מתקדם שמשתמש במודל AI המאומן
class AITalmudNikudProcessor(SmartPDFNikudProcessor):
    """מעבד ניקוד מתקדם עם מודל AI אמיתי"""

    def __init__(self):
        super().__init__()
        self.ai_model = None
        self.ai_tokenizer = None
        self.label_mapping = None
        self.processed_words = 0
        self.successful_words = 0
        self.ai_successful_words = 0

        # מילון גיבוי
        self.backup_dict = {
            'שלושה': 'שְׁלוֹשָׁה', 'ימים': 'יָמִים', 'אסור': 'אָסוּר',
            'לשאת': 'לָשֵׂאת', 'ולתת': 'וְלָתֵת', 'עמו': 'עִמּוֹ',
            'דכתיב': 'דִּכְתִיב', 'דאמר': 'דַּאֲמַר', 'מישום': 'מִשּׁוּם',
            'רבי': 'רַבִּי', 'אמר': 'אָמַר', 'יהודה': 'יְהוּדָה'
        }

    def load_ai_model(self, model_path='./nikud_model_final'):
        """טעינת המודל המאומן"""
        try:
            from transformers import AutoTokenizer, AutoModelForTokenClassification
            import json

            self.ai_tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.ai_model = AutoModelForTokenClassification.from_pretrained(model_path)

            with open(f'{model_path}/label_mapping.json', 'r', encoding='utf-8') as f:
                self.label_mapping = json.load(f)

            print("✅ מודל AI נטען בהצלחה!")
            print(f"📊 המודל מכיר {len(self.label_mapping['label_to_id'])} סוגי ניקוד")
            return True

        except Exception as e:
            print(f"⚠️ כשל בטעינת מודל AI: {e}")
            print("🔄 נמשיך עם מילון גיבוי")
            return False

    def predict_nikud_with_ai(self, sentence):
        """ניקוד משפט באמצעות המודל האמיתי עם זיהוי ספקות"""
        if not self.ai_model or not self.ai_tokenizer:
            return None, None

        try:
            import torch
            inputs = self.ai_tokenizer(sentence, return_tensors='pt', max_length=128, truncation=True, padding=True)

            with torch.no_grad():
                outputs = self.ai_model(**inputs)
                probabilities = torch.softmax(outputs.logits, dim=-1)
                predictions = torch.argmax(outputs.logits, dim=-1)

                uncertainties = []
                for i, prob_dist in enumerate(probabilities[0]):
                    sorted_probs, sorted_indices = torch.sort(prob_dist, descending=True)
                    if len(sorted_probs) >= 2:
                        best_prob = sorted_probs[0].item()
                        second_prob = sorted_probs[1].item()
                        if best_prob - second_prob < 0.2 and best_prob > 0.3:
                            uncertainties.append({
                                'position': i,
                                'char': sentence[i] if i < len(sentence) else '',
                                'best_option': self.label_mapping['id_to_label'].get(str(sorted_indices[0].item()), 'UNKNOWN'),
                                'second_option': self.label_mapping['id_to_label'].get(str(sorted_indices[1].item()), 'UNKNOWN'),
                                'confidence_gap': best_prob - second_prob,
                                'best_confidence': best_prob
                            })

            predicted_labels = predictions[0].tolist()
            id_to_label = self.label_mapping['id_to_label']
            result = ""

            for i, char in enumerate(sentence):
                result += char
                if i < len(predicted_labels):
                    label_id = predicted_labels[i]
                    label = id_to_label.get(str(label_id), '')
                    if label not in ['NO_NIKUD', 'SPACE', 'OTHER'] and len(label) <= 3:
                        result += label

            return result, uncertainties

        except Exception as e:
            print(f"⚠️ שגיאה בחיזוי AI: {e}")
            return None, None

    def nikud_sentence_smart(self, sentence):
        """ניקוד משפט עם AI + גיבוי וזיהוי ספקות"""
        self.processed_words += 1

        if self.ai_model:
            ai_result, uncertainties = self.predict_nikud_with_ai(sentence)
            if ai_result and self.is_valid_nikud(sentence, ai_result):
                self.ai_successful_words += 1
                self.successful_words += 1
                if uncertainties:
                    self.add_uncertainty_case(sentence, ai_result, uncertainties)
                return ai_result

        words = sentence.split()
        nikud_words = []
        for word in words:
            clean_word = self.remove_existing_nikud(word)
            if clean_word in self.backup_dict:
                nikud_words.append(self.backup_dict[clean_word])
                self.successful_words += 1
            else:
                nikud_words.append(word)

        return ' '.join(nikud_words)

    def add_uncertainty_case(self, original_sentence, nikud_result, uncertainties):
        """הוספת מקרה ספק לרשימה"""
        if not hasattr(self, 'uncertainty_cases'):
            self.uncertainty_cases = []

        alternatives = []
        for u in uncertainties:
            if u['best_option'] != u['second_option']:
                alternatives.append({
                    'position': u['position'],
                    'character': u['char'],
                    'chosen': u['best_option'],
                    'alternative': u['second_option'],
                    'confidence': f"{u['best_confidence']:.1%}",
                    'gap': f"{u['confidence_gap']:.1%}"
                })

        if alternatives:
            self.uncertainty_cases.append({
                'id': len(self.uncertainty_cases) + 1,
                'original': original_sentence,
                'result': nikud_result,
                'alternatives': alternatives,
                'context': self.get_sentence_context(original_sentence)
            })

    def get_sentence_context(self, sentence):
        """זיהוי הקשר המשפט (הלכה, פסוק, דין וכו')"""
        sentence_lower = sentence.lower()
        if any(word in sentence_lower for word in ['דכתיב', 'כתיב', 'שנאמר']):
            return 'פסוק/מקרא'
        elif any(word in sentence_lower for word in ['רבי', 'אמר', 'סבר']):
            return 'אמירת חכם'
        elif any(word in sentence_lower for word in ['הלכה', 'פסק', 'דין']):
            return 'הלכה למעשה'
        elif any(word in sentence_lower for word in ['דתניא', 'תנו רבנן', 'משנה']):
            return 'מקור תנאי'
        elif any(word in sentence_lower for word in ['אמרינן', 'גמרא', 'בעיא']):
            return 'דיון גמרא'
        return 'כללי'

    def export_uncertainty_report(self, output_file="uncertainty_report.txt"):
        """ייצוא דו"ח ספקות לבדיקה ידנית"""
        if not hasattr(self, 'uncertainty_cases') or not self.uncertainty_cases:
            print("📋 אין מקרי ספק לייצוא")
            return False

        try:
            from datetime import datetime
            import json

            report = [
                "דו\"ח מקרי ספק בניקוד התלמוד הבבלי",
                "=========================================",
                "",
                f"📊 סה\"כ מקרי ספק: {len(self.uncertainty_cases)}",
                f"🤖 מתוך {self.processed_words} משפטים",
                f"📈 אחוז מקרי ספק: {(len(self.uncertainty_cases) / self.processed_words * 100):.1f}%",
                "",
                "================ פירוט ================="
            ]

            for case in self.uncertainty_cases:
                report.append(f"\n🔹 מקרה #{case['id']}")
                report.append(f"📝 משפט: {case['original']}")
                report.append(f"🎯 תוצאה: {case['result']}")
                report.append(f"📚 הקשר: {case['context']}")
                report.append("🤔 ספקות:")
                for alt in case['alternatives']:
                    char = alt['character'] or f"(מיקום {alt['position']})"
                    report.append(f"  • {char}: נבחר {alt['chosen']} ({alt['confidence']}), חלופה: {alt['alternative']} (פער: {alt['gap']})")

            # סיכום לפי הקשר
            context_summary = {}
            for case in self.uncertainty_cases:
                context_summary[case['context']] = context_summary.get(case['context'], 0) + 1

            report.append("\nסיכום לפי סוגי הקשר:")
            for context, count in sorted(context_summary.items(), key=lambda x: x[1], reverse=True):
                pct = (count / len(self.uncertainty_cases)) * 100
                report.append(f"📊 {context}: {count} מקרים ({pct:.1f}%)")

            report.append(f"\n📅 תאריך הדו\"ח: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

            with open(output_file, 'w', encoding='utf-8') as f:
                f.write('\n'.join(report))

            with open(output_file.replace('.txt', '.json'), 'w', encoding='utf-8') as jf:
                json.dump(self.uncertainty_cases, jf, ensure_ascii=False, indent=2)

            print(f"✅ דו\"ח נוצר: {output_file}")
            return True

        except Exception as e:
            print(f"❌ שגיאה ביצירת הדו\"ח: {e}")
            return False

    def is_valid_nikud(self, original, nikud_result):
        """בדיקת תקינות תוצאה"""
        if not nikud_result or len(nikud_result) < len(original):
            return False

        return self.remove_existing_nikud(nikud_result) == self.remove_existing_nikud(original)

    def process_text_with_ai(self, text):
        """עיבוד טקסט מלא עם AI"""
        print("🤖 מעבד טקסט עם מודל AI...")
        segments = self.smart_text_segmentation(text)
        hebrew_sentences, current_sentence = [], ""

        for segment in segments:
            if segment['needs_nikud']:
                current_sentence += segment['content']
            else:
                if current_sentence.strip():
                    hebrew_sentences.append(current_sentence.strip())
                    current_sentence = ""

        if current_sentence.strip():
            hebrew_sentences.append(current_sentence.strip())

        nikud_sentences = {}
        for sentence in hebrew_sentences:
            if len(sentence) > 3:
                nikud_sentences[sentence] = self.nikud_sentence_smart(sentence)

        processed_text, current_sentence = "", ""
        for segment in segments:
            if segment['needs_nikud']:
                current_sentence += segment['content']
            else:
                if current_sentence.strip() in nikud_sentences:
                    processed_text += nikud_sentences[current_sentence.strip()]
                else:
                    processed_text += current_sentence
                current_sentence = ""
                processed_text += segment['content']

        if current_sentence.strip() in nikud_sentences:
            processed_text += nikud_sentences[current_sentence.strip()]
        else:
            processed_text += current_sentence

        return processed_text

    def get_ai_statistics(self):
        """סטטיסטיקות מפורטות כולל AI ומקרי ספק"""
        if self.processed_words == 0:
            return {
                'processed': 0,
                'ai_success_rate': '0%',
                'total_success': '0%',
                'uncertainties': 0
            }

        return {
            'processed_sentences': self.processed_words,
            'ai_successful': self.ai_successful_words,
            'ai_success_rate': f"{(self.ai_successful_words / self.processed_words) * 100:.1f}%",
            'total_success_rate': f"{(self.successful_words / self.processed_words) * 100:.1f}%",
            'fallback_used': self.successful_words - self.ai_successful_words,
            'uncertainty_cases': len(getattr(self, 'uncertainty_cases', [])),
            'uncertainty_rate': f"{(len(getattr(self, 'uncertainty_cases', [])) / self.processed_words) * 100:.1f}%"
        }

        extended_test = """
        Q0@79@.ב99@

        38@
        29@
        @06לפני @16איזהו שלושה ימים אסור לשאת ולתת עמו. @23עובדי עבודה זרה: @13שלשה ימים אסור
        לשאת ולתת עמו. @23גכולהו @23מישום דאיל ומודה @23לעבודה זרת ביום אידו.

        רבי יהודה אמר דכתיב ולא יעבוד עבודה זרה ביום אידו.
        אמר רב אשי הכי קאמר קרא ולא יעבוד.
        דתניא אליעזר אומר אסור לשאת ולתת עם עובדי כוכבים.
        """

        print("🔄 מעבד טקסט עם זיהוי ספקות...")

        # עיבוד הטקסט
        result_text = ai_processor.process_text_with_ai(extended_test)

        # הצגת סטטיסטיקות מפורטות
        stats = ai_processor.get_ai_statistics()
        print("\n📊 סטטיסטיקות עיבוד מפורטות:")
        print(f"משפטים שעובדו: {stats['processed_sentences']}")
        print(f"הצלחה עם AI: {stats['ai_success_rate']}")
        print(f"הצלחה כוללת: {stats['total_success_rate']}")
        print(f"מקרי ספק זוהו: {stats['uncertainty_cases']} ({stats['uncertainty_rate']})")

        # ייצוא דו"ח ספקות
        if stats['uncertainty_cases'] > 0:
            print(f"\n🤔 זוהו {stats['uncertainty_cases']} מקרי ספק!")
            success = ai_processor.export_uncertainty_report("talmud_uncertainties.txt")

            if success:
                print("📋 דו\"ח ספקות נוצר - עברי עליו ובדקי את הבחירות")

                # הורדת הדו"ח ב-Colab
                from google.colab import files
                try:
                    files.download("talmud_uncertainties.txt")
                    files.download("talmud_uncertainties.json")
                    print("📥 קבצי הדו\"ח הורדו למחשב שלך")
                except:
                    print("⚠️ לא הצלחתי להוריד אוטומטית")
        else:
            print("✅ לא נמצאו מקרי ספק - המודל בטוח בכל הבחירות")

        # הצגת דוגמה מהתוצאה
        print("\n📝 דוגמה מהתוצאה:")
        print("-" * 50)
        print(result_text[:400] + "...")
        print("-" * 50)




    def process_pdf_to_nikud(self, pdf_path):
        """עיבוד PDF מלא לניקוד עם שימור מבנה"""
        print("🎯 מתחיל עיבוד PDF מלא...")

        # חילוץ טקסט
        raw_text = self.extract_pdf_with_structure(pdf_path)
        if not raw_text:
            print("❌ לא הצלחתי לחלץ טקסט מה-PDF")
            return None

        print(f"📝 חולץ {len(raw_text)} תווים")

        # עיבוד הטקסט לניקוד
        return self.process_text_to_nikud(raw_text)


    def extract_pdf_with_structure(self, pdf_path):
        print(f"📖 מחלץ טקסט מ-PDF: {pdf_path}")
        try:
            doc = fitz.open(pdf_path)
            page = doc[0]
            text = page.get_text("text")
            text_dict = page.get_text("dict")
            return text

        except Exception as e:
            print(f"⚠️ נכשל pdfplumber, מנסה PyPDF2: {e}")
            try:
                with open(pdf_path, 'rb') as file:
                    reader = PyPDF2.PdfReader(file)
                    all_text = ""
                    for page_num, page in enumerate(reader.pages, 1):
                        page_text = page.extract_text()
                        if page_text:
                            all_text += page_text + "\n\n"
                    return all_text
            except Exception as e2:
                print(f"❌ כשל גם PyPDF2: {e2}")
                return None

    def process_text_to_nikud(self, text):
        """עיבוד טקסט לניקוד עם שימור מבנה מלא"""
        # פיצול הטקסט לסגמנטים תוך שימור העיצוב
        segments = self.smart_text_segmentation(text)

        processed_text = ""
        hebrew_words_processed = 0

        for segment in segments:
          if segment['needs_nikud']:
            # זוהתה מילה עברית - צריך ניקוד
            try:
                nikud_word = self.add_nikud_to_hebrew_word(segment['content'])
                processed_text += nikud_word
                hebrew_words_processed += 1

                # דיווח התקדמות
                if hebrew_words_processed % 50 == 0:
                  print(f"✅ עובד על מילה עברית {hebrew_words_processed}")

            except Exception as e:
              # במקרה של שגיאה - שמור את המילה כמו שהיא
              processed_text += segment['content']
              print(f"⚠️ שגיאה בעיבוד '{segment['content']}': {e}")
          else:
            # סימני עיצוב, רווחים, פיסוק - שמור כמו שהם
            processed_text += segment['content']
            print(f"🎉 הושלם עיבוד {hebrew_words_processed} מילים עבריות")

        return processed_text

    def add_nikud_to_hebrew_word(self, word):
        """הוספת ניקוד למילה עברית בודדת"""

        # הסרת ניקוד קיים
        talmud_training_data_generator = TalmudTrainingDataGenerator()
        clean_word = talmud_training_data_generator.remove_nikud(word)

        # בסיס נתונים לניקוד - מילים נפוצות בתלמוד
        nikud_database = {
            # מילים בסיסיות בתלמוד
            'רבי': 'רַבִּי',
            'אמר': 'אָמַר',
            'אומר': 'אוֹמֵר',
            'דכתיב': 'דִּכְתִיב',
            'הכי': 'הָכִי',
            'קאמר': 'קָאָמַר',
            'דאמר': 'דְּאָמַר',
            'תניא': 'תַּנְיָא',
            'דתניא': 'דְּתַנְיָא',
            'הוא': 'הוּא',
            'היא': 'הִיא',
            'זה': 'זֶה',
            'זו': 'זוּ',
            'כל': 'כָּל',
            'על': 'עַל',
            'אל': 'אֶל',
            'מן': 'מִן',
            'אין': 'אֵין',
            'יש': 'יֵשׁ',
            'לא': 'לֹא',
            'כן': 'כֵּן',
            'גם': 'גַּם',
            'רק': 'רַק',
            'אך': 'אַךְ',
            'או': 'אוֹ',

            # מילים דתיות נפוצות
            'תורה': 'תּוֹרָה',
            'מצוה': 'מִצְוָה',
            'מצות': 'מִצְווֹת',
            'הלכה': 'הֲלָכָה',
            'משנה': 'מִשְׁנָה',
            'תלמוד': 'תַּלְמוּד',
            'ברכה': 'בְּרָכָה',
            'ברכות': 'בְּרָכוֹת',
            'שבת': 'שַׁבָּת',
            'יום': 'יוֹם',
            'לילה': 'לַיְלָה',
            'בוקר': 'בֹּקֶר',
            'ערב': 'עֶרֶב',

            # פעלים נפוצים
            'אכל': 'אָכַל',
            'שתה': 'שָׁתָה',
            'הלך': 'הָלַךְ',
            'בא': 'בָּא',
            'יצא': 'יָצָא',
            'נתן': 'נָתַן',
            'לקח': 'לָקַח',
            'שמע': 'שָׁמַע',
            'ראה': 'רָאָה',
            'עשה': 'עָשָׂה',
            'דבר': 'דִּבֵּר',
            'קרא': 'קָרָא',

            # שמות עצם נפוצים
            'בית': 'בַּיִת',
            'ספר': 'סֵפֶר',
            'דבר': 'דָּבָר',
            'דברים': 'דְּבָרִים',
            'איש': 'אִישׁ',
            'אשה': 'אִשָּׁה',
            'בן': 'בֵּן',
            'בת': 'בַּת',
            'אב': 'אָב',
            'אם': 'אֵם',
            'מלך': 'מֶלֶךְ',
            'עם': 'עַם',
            'ארץ': 'אֶרֶץ',
            'שמים': 'שָׁמַיִם',
            'מים': 'מַיִם',
            'אש': 'אֵשׁ',
            'רוח': 'רוּחַ',
            'נפש': 'נֶפֶשׁ',
            'לב': 'לֵב',
            'עין': 'עַיִן',
            'יד': 'יָד',
            'רגל': 'רֶגֶל',
            'ראש': 'רֹאשׁ',
            'פה': 'פֶּה',

            # שמות תנאים ואמוראים נפוצים
            'יהודה': 'יְהוּדָה',
            'שמעון': 'שִׁמְעוֹן',
            'יוסי': 'יוֹסֵי',
            'מאיר': 'מֵאִיר',
            'עקיבא': 'עֲקִיבָא',
            'אליעזר': 'אֱלִיעֶזֶר',
            'יהושע': 'יְהוֹשֻׁעַ',
            'אבא': 'אַבָּא',
            'רבא': 'רָבָא',
            'אביי': 'אַבַּיֵי',
            'אשי': 'אָשֵׁי',

            # מילות יחס וקשר
            'של': 'שֶׁל',
            'את': 'אֶת',
            'עם': 'עִם',
            'בין': 'בֵּין',
            'אחר': 'אַחַר',
            'לפני': 'לִפְנֵי',
            'אחרי': 'אַחֲרֵי',
            'תחת': 'תַּחַת',
            'אצל': 'אֵצֶל'
        }

        # 1. חיפוש ישיר במילון
        if clean_word in nikud_database:
            return nikud_database[clean_word]

        # 2. טיפול בקידומות נפוצות
        if len(clean_word) > 1:
            first_char = clean_word[0]
            rest_of_word = clean_word[1:]

            # ו' החיבור
            if first_char == 'ו' and rest_of_word in nikud_database:
                base_nikud = nikud_database[rest_of_word]
                # ו' לפני בגדכפת = וּ, אחרת = וְ
                if rest_of_word[0] in 'בגדכפת':
                    return 'וּ' + base_nikud
                else:
                    return 'וְ' + base_nikud

            # ה' הידיעה
            elif first_char == 'ה' and rest_of_word in nikud_database:
                base_nikud = nikud_database[rest_of_word]
                # ה' לפני אותיות גרון = הָ, אחרת = הַ
                if rest_of_word[0] in 'אהחע':
                    return 'הָ' + base_nikud
                else:
                    return 'הַ' + base_nikud

            # כ' ההשוואה/התיאור
            elif first_char == 'כ' and rest_of_word in nikud_database:
                base_nikud = nikud_database[rest_of_word]
                return 'כְּ' + base_nikud

            # ל' הכיוון
            elif first_char == 'ל' and rest_of_word in nikud_database:
                base_nikud = nikud_database[rest_of_word]
                return 'לְ' + base_nikud

            # מ' המקום/התחלה
            elif first_char == 'מ' and rest_of_word in nikud_database:
                base_nikud = nikud_database[rest_of_word]
                return 'מִ' + base_nikud

            # ב' המקום
            elif first_char == 'ב' and rest_of_word in nikud_database:
                base_nikud = nikud_database[rest_of_word]
                return 'בְּ' + base_nikud

        # 3. טיפול בסופיות נפוצות
        suffix_patterns = {
            'ים': ('ִים', 2),    # רבים זכר
            'ות': ('וֹת', 2),    # רבים נקבה
            'תי': ('תִּי', 2),    # גוף ראשון יחיד
            'נו': ('נוּ', 2),     # גוף ראשון רבים
            'תם': ('תֶּם', 2),   # גוף שני רבים זכר
            'תן': ('תֶּן', 2),   # גוף שני רבים נקבה
            'ה': ('ָה', 1)       # נקבה או כיוון
        }

        for suffix, (nikud_suffix, suffix_len) in suffix_patterns.items():
            if clean_word.endswith(suffix) and len(clean_word) > suffix_len:
                base_word = clean_word[:-suffix_len]
                if base_word in nikud_database:
                    return nikud_database[base_word] + nikud_suffix

        # 4. ניקוד בסיסי למילים לא מוכרות
        return self.basic_nikud_fallback(clean_word)

    def basic_nikud_fallback(self, word):
        """ניקוד בסיסי למילים לא מוכרות"""

        if len(word) <= 2:
            return word

        # אלגוריתם פשוט: שווא לרוב האותיות הפנימיות
        nikud_word = word[0]  # האות הראשונה ללא ניקוד

        for i in range(1, len(word)):
            char = word[i]

            # לא להוסיף ניקוד לאות האחרונה
            if i == len(word) - 1:
                nikud_word += char
            else:
                # הוספת שווא לאותיות שאינן אמ"ה
                if char in 'אהוי':  # אותיות אמ"ה
                    nikud_word += char
                else:
                    nikud_word += char + '\u05B0'  # שווא נע

        return nikud_word



In [15]:
class SmartPDFNikudProcessor:
    """מעבד PDF חכם לניקוד התלמוד הבבלי - גרסה מורחבת"""

    def __init__(self):
        print("🚀 מפעיל מעבד PDF חכם לתלמוד (גרסה מורחבת)...")

    def is_formatting_element(self, text):
        """בדיקה אם הטקסט הוא אלמנט עיצוב שצריך לשמור"""
        text = text.strip()

        if not text:
            return False

        # זיהוי סימני התלמוד
        if re.match(r'Q\d+@', text):  # Q0@, Q1@
            return True
        if re.match(r'\d+@', text):  # 79@, 38@, 100@
            return True
        if re.match(r'@\d+', text):  # @23, @16, @13
            return True
        if re.match(r'\.ב\d+@', text):  # .ב99@
            return True
        if re.match(r'\d+:\d+@', text):  # 420:23@
            return True
        if re.match(r'@\d+@', text):  # @700@, @350@
            return True
        if text in ['.', ',', ':', ';', '(', ')', '[', ']', '{', '}', '•', '·']:
            return True
        if len(text) <= 2 and not any('\u0590' <= c <= '\u05FF' for c in text):
            return True

        return False

    def smart_text_segmentation(self, text):
        """חלוקה חכמה של הטקסט לחלקים"""
        segments = []
        lines = text.split('\n')

        for line in lines:
            if not line.strip():
                segments.append({
                    'type': 'empty_line',
                    'content': line,
                    'needs_nikud': False
                })
                continue

            # פיצול מתקדם
            pattern = r'(Q\d+@[\d@.ב]*|\d+@|@\d+|\.ב\d+@|\d+:\d+@|@\d+@|\([^)]*\)|[\u0590-\u05FF]+|[^\u0590-\u05FF\s]+|\s+)'
            parts = re.findall(pattern, line)

            # גיבוי אם הרגקס כושל
            if not parts or ''.join(parts) != line:
                words = re.split(r'(\s+)', line)
                parts = [word for word in words if word]

            for part in parts:
                if not part:
                    continue

                if part.isspace():
                    segments.append({
                        'type': 'space',
                        'content': part,
                        'needs_nikud': False
                    })
                elif self.is_formatting_element(part):
                    segments.append({
                        'type': 'formatting',
                        'content': part,
                        'needs_nikud': False
                    })
                elif any('\u0590' <= c <= '\u05FF' for c in part):
                    segments.append({
                        'type': 'hebrew',
                        'content': part,
                        'needs_nikud': True
                    })
                else:
                    segments.append({
                        'type': 'other',
                        'content': part,
                        'needs_nikud': False
                    })

        return segments

    def remove_existing_nikud(self, text):
        """הסרת ניקוד קיים מהטקסט"""
        nikud_chars = '\u05B0\u05B1\u05B2\u05B3\u05B4\u05B5\u05B6\u05B7\u05B8\u05B9\u05BA\u05BB\u05BC\u05BD\u05BE\u05BF\u05C0\u05C1\u05C2\u05C3\u05C4\u05C5\u05C6\u05C7'
        return ''.join(char for char in text if char not in nikud_chars)

    def extract_pdf_with_structure(self, pdf_path):
        """חילוץ טקסט מ-PDF עם שימור מבנה"""
        print(f"📖 מחלץ טקסט מ-PDF: {pdf_path}")

        try:
            doc = fitz.open(pdf_path)
            page = doc[0]
            text = page.get_text("text")
            text_dict = page.get_text("dict")
            return text
        except Exception as e:
            print(f"⚠️ נכשל pdfplumber, מנסה PyPDF2: {e}")

            # ניסיון חלופי עם PyPDF2
            try:
                with open(pdf_path, 'rb') as file:
                    reader = PyPDF2.PdfReader(file)
                    all_text = ""
                    for page_num, page in enumerate(reader.pages, 1):
                        page_text = page.extract_text()
                        if page_text:
                            all_text += page_text + "\n\n"
                    return all_text
            except Exception as e2:
                print(f"❌ כשל גם PyPDF2: {e2}")
                return None

    def save_to_word(self, text, output_filename="talmud_nikud.docx"):
        """שמירה לקובץ Word עם עיצוב נכון"""
        print(f"💾 שומר לקובץ Word: {output_filename}")

        try:
            doc = Document()

            # הגדרת עיצוב בסיסי
            style = doc.styles['Normal']
            font = style.font
            font.name = 'David'
            font.size = Pt(12)

            # פיצול לפסקאות
            paragraphs = text.split('\n\n')

            for para_text in paragraphs:
                if para_text.strip():
                    paragraph = doc.add_paragraph()

                    if '@' in para_text:
                        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                        run = paragraph.add_run(para_text.strip())
                        run.bold = True
                    else:
                        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
                        paragraph.add_run(para_text.strip())
                else:
                    doc.add_paragraph()

            doc.save(output_filename)
            print(f"✅ נשמר בהצלחה: {output_filename}")
            return True

        except Exception as e:
            print(f"❌ שגיאה בשמירה: {e}")
            return False

# יצירת המעבד המורחב החדש
processor = SmartPDFNikudProcessor()

# בדיקה שכל הפונקציות עובדות
print("🔍 בדיקת פונקציות חדשות:")

# בדיקת הסרת ניקוד
test_nikud = "שָׁלוֹם עוֹלָם"
clean_text = processor.remove_existing_nikud(test_nikud)
print(f"טקסט עם ניקוד: {test_nikud}")
print(f"טקסט נקי: {clean_text}")

# בדיקת זיהוי מחדש
result = processor.smart_text_segmentation(test_text)
print(f"מספר חלקים שזוהו: {len(result)}")



🚀 מפעיל מעבד PDF חכם לתלמוד (גרסה מורחבת)...
🔍 בדיקת פונקציות חדשות:
טקסט עם ניקוד: שָׁלוֹם עוֹלָם
טקסט נקי: שלום עולם
מספר חלקים שזוהו: 40


In [29]:


class TalmudNikudProcessor(SmartPDFNikudProcessor):
    """
    מעבד מתקדם לניקוד התלמוד הבבלי
    """

    def __init__(self):
        super().__init__()
        self.trained_model = None
        self.nikud_pipeline = None
        self.success_rate = 0
        self.processed_words = 0
        self.successful_words = 0

    def load_trained_model(self, model_path='./nikud_model_trained'):
        """טעינת המודל המאומן"""
        try:
            from transformers import pipeline
            model_path = "./nikud_model_final"
            self.nikud_pipeline = pipeline(
                'token-classification',
                model=model_path,
                tokenizer=model_path,
                aggregation_strategy="simple"
            )
            print("✅ מודל מאומן נטען בהצלחה")
            return True
        except Exception as e:
            print(f"⚠️ כשל בטעינת מודל מאומן: {e}")
            print("🔄 משתמש במילון סטטי")
            return False


    def add_nikud_to_hebrew_word(self, word):
        """ניקוד מילה עם המודל המאומן או מילון"""
        self.processed_words += 1

        try:
            # ניסיון עם המודל המאומן
            if self.nikud_pipeline:
                result = self.nikud_pipeline(
                    word,
                    max_length=len(word) + 20,
                    do_sample=False
                )[0]['generated_text']

                # בדיקת תקינות התוצאה
                if self.is_valid_nikud_result(word, result):
                    self.successful_words += 1
                    return result

            # חזרה למילון כגיבוי
            result = advanced_nikud_word(word, talmud_nikud_dict)
            if result != word:  # אם נמצא במילון
                self.successful_words += 1

            return result

        except Exception as e:
            print(f"⚠️ שגיאה בניקוד '{word}': {e}")
            return word

    def is_valid_nikud_result(self, original, nikud_result):
        """בדיקת תקינות תוצאת הניקוד"""
        # בדיקות בסיסיות
        if len(nikud_result) < len(original):
            return False

        # הסרת ניקוד ובדיקה שהמילה זהה
        clean_result = self.remove_existing_nikud(nikud_result)
        clean_original = self.remove_existing_nikud(original)

        return clean_result == clean_original

    def get_success_statistics(self):
        """קבלת סטטיסטיקות הצלחה"""
        if self.processed_words == 0:
            return "טרם עובד על מילים"

        success_rate = (self.successful_words / self.processed_words) * 100

        return {
            'processed_words': self.processed_words,
            'successful_words': self.successful_words,
            'success_rate': f"{success_rate:.1f}%",
            'failed_words': self.processed_words - self.successful_words
        }

# יצירת המעבד החדש
processor = TalmudNikudProcessor()
processor.load_trained_model()

🚀 מפעיל מעבד PDF חכם לתלמוד (גרסה מורחבת)...


Device set to use cpu


✅ מודל מאומן נטען בהצלחה


True

In [28]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch
import numpy as np

class NikudDataset(Dataset):
    """Dataset מותאם לאימון ניקוד"""

    def __init__(self, examples, tokenizer, max_length=128):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_length = max_length

        # יצירת מפת תוויות
        all_labels = set()
        for example in examples:
            all_labels.update(example['labels'])

        self.label_to_id = {label: idx for idx, label in enumerate(sorted(all_labels))}
        self.id_to_label = {idx: label for label, idx in self.label_to_id.items()}

        print(f"📋 תוויות ניקוד זוהו: {len(self.label_to_id)}")
        print(f"דוגמאות: {list(self.label_to_id.keys())[:10]}")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]

        # טוקניזציה
        encoding = self.tokenizer(
            example['text'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # התאמת תוויות לטוקנים
        labels = [-100] * self.max_length  # -100 = ignore

        # מיפוי פשוט - לכל תו תווית
        for i, label in enumerate(example['labels'][:self.max_length-2]):  # השאר מקום ל-CLS ו-SEP
            if label in self.label_to_id:
                labels[i+1] = self.label_to_id[label]  # +1 בגלל CLS

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

# הכנת הנתונים לאימון
if available_models and training_data:
    model_name = available_models[0]
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # יצירת dataset
    dataset = NikudDataset(training_data, tokenizer)

    # טעינת מודל
    model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=len(dataset.label_to_id),
        ignore_mismatched_sizes=True
    )

    # הגדרות אימון
    training_args = TrainingArguments(
        output_dir='./nikud_model_trained',
        num_train_epochs=10,  # אימון יותר ארוך למודל טוב
        per_device_train_batch_size=4,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        save_steps=100,
        # evaluation_strategy="no",
        learning_rate=3e-5,
        save_total_limit=2
    )

    # יצירת Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer
    )

    print("🚀 מתחיל אימון מודל AI אמיתי...")
    print("⏱️ זה יקח כ-10-15 דקות...")

    import os
    os.environ["WANDB_DISABLED"] = "true"
    # אימון
    trainer.train()

    # שמירת המודל
    model.save_pretrained('./nikud_model_final')
    tokenizer.save_pretrained('./nikud_model_final')

    # שמירת מיפוי התוויות
    import json
    with open('./nikud_model_final/label_mapping.json', 'w', encoding='utf-8') as f:
        json.dump({
            'label_to_id': dataset.label_to_id,
            'id_to_label': dataset.id_to_label
        }, f, ensure_ascii=False, indent=2)

    print("✅ אימון הושלם והמודל נשמר!")

else:
    print("⚠️ לא ניתן לאמן מודל - נמשיך עם מילון")

📋 תוויות ניקוד זוהו: 20
דוגמאות: ['NO_NIKUD', 'SPACE', 'ְ', 'ְּ', 'ְׁ', 'ֱ', 'ֲ', 'ִ', 'ִּ', 'ֵ']


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-3941512201.py:86: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 מתחיל אימון מודל AI אמיתי...
⏱️ זה יקח כ-10-15 דקות...


Step,Training Loss
10,2.945100
20,2.720600


✅ אימון הושלם והמודל נשמר!


In [12]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

models_to_check = [
    'dicta-il/dictabert',      # המודל הטוב ביותר לעברית
    'avichr/heBERT',           # חלופה
    'onlplab/alephbert-base'   # עוד חלופה
]

available_models = []
for model_name in models_to_check:
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        print(f"✅ {model_name} - זמין")
        available_models.append(model_name)

        # בדיקה בסיסית
        test_tokens = tokenizer.tokenize("רבי יהודה אמר")
        print(f"   דוגמת טוקניזציה: {test_tokens}")

    except Exception as e:
        print(f"❌ {model_name} - לא זמין: {e}")

if not available_models:
    print("⚠️ אין מודלים זמינים - נשתמש במילון בלבד")
else:
    print(f"🎯 נשתמש במודל: {available_models[0]}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ dicta-il/dictabert - זמין
   דוגמת טוקניזציה: ['רבי', 'יהודה', 'אמר']


...

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

✅ onlplab/alephbert-base - זמין
   דוגמת טוקניזציה: ['רבי', 'יהודה', 'אמר']
🎯 נשתמש במודל: dicta-il/dictabert


In [13]:
class TalmudTrainingDataGenerator:
    """יוצר נתוני אימון לניקוד עם הקשר"""

    def __init__(self):
        # משפטים מלאים מהתלמוד עם ניקוד נכון
        self.training_sentences = [
            {
                'without_nikud': 'רבי יהודה אמר דכתיב שלושה ימים',
                'with_nikud': 'רַבִּי יְהוּדָה אָמַר דִּכְתִיב שְׁלוֹשָׁה יָמִים',
                'context': 'הלכה'
            },
            {
                'without_nikud': 'אמר רב אשי הכי קאמר',
                'with_nikud': 'אָמַר רַב אָשֵׁי הָכִי קָאָמַר',
                'context': 'דין'
            },
            {
                'without_nikud': 'ועוד דתניא במשנה',
                'with_nikud': 'וְעוֹד דְּתַנְיָא בַּמִּשְׁנָה',
                'context': 'מקור'
            },
            {
                'without_nikud': 'דכתיב ולא יעבוד עבודה זרה',
                'with_nikud': 'דִּכְתִיב וְלֹא יַעֲבוֹד עֲבוֹדָה זָרָה',
                'context': 'פסוק'
            },
            {
                'without_nikud': 'איכא למימר דהכי קאמר',
                'with_nikud': 'אִיכָּא לְמֵימַר דְּהָכִי קָאָמַר',
                'context': 'טיעון'
            },
            {
                'without_nikud': 'משום דאמר רב פפא',
                'with_nikud': 'מִשּׁוּם דְּאָמַר רַב פָּפָּא',
                'context': 'סברא'
            },
            {
                'without_nikud': 'והא דתניא אליעזר אומר',
                'with_nikud': 'וְהָא דְּתַנְיָא אֱלִיעֶזֶר אוֹמֵר',
                'context': 'מחלוקת'
            },
            {
                'without_nikud': 'בעבודה זרה ביום אידו אסור',
                'with_nikud': 'בַּעֲבוֹדָה זָרָה בְּיוֹם אִידוּ אָסוּר',
                'context': 'הלכה'
            },
            {
                'without_nikud': 'רבנן סברי הכי ורבי יהודה סבר הכי',
                'with_nikud': 'רַבָּנַן סְבִירֵי הָכִי וְרַבִּי יְהוּדָה סָבַר הָכִי',
                'context': 'מחלוקת'
            },
            {
                'without_nikud': 'ואמרינן בגמרא דהלכה כרבי יהודה',
                'with_nikud': 'וְאָמְרִינַן בַּגְּמָרָא דְּהִלְכָתָא כְּרַבִּי יְהוּדָה',
                'context': 'פסיקה'
            }
        ]

    def create_token_labels(self, sentence_without, sentence_with):
        """יוצר תוויות לכל טוקן במשפט"""

        # הסרת ניקוד מהמשפט המנוקד לצורך השוואה
        clean_nikud = self.remove_nikud(sentence_with)

        if sentence_without != clean_nikud:
            print(f"⚠️ אי-התאמה: '{sentence_without}' vs '{clean_nikud}'")
            return None

        # יצירת תוויות לכל תו
        labels = []
        nikud_idx = 0

        for char in sentence_without:
            if char == ' ':
                labels.append('SPACE')
                nikud_idx += 1
            elif '\u0590' <= char <= '\u05FF':  # תו עברי
                # מציאת הניקוד המתאים
                if nikud_idx < len(sentence_with) and char == sentence_with[nikud_idx]:
                    nikud_idx += 1

                    # בדיקה אם יש ניקוד אחרי התו
                    nikud_found = []
                    while (nikud_idx < len(sentence_with) and
                           '\u05B0' <= sentence_with[nikud_idx] <= '\u05C7'):
                        nikud_found.append(sentence_with[nikud_idx])
                        nikud_idx += 1

                    if nikud_found:
                        labels.append(''.join(nikud_found))
                    else:
                        labels.append('NO_NIKUD')
                else:
                    labels.append('NO_NIKUD')
            else:
                labels.append('OTHER')

        return labels

    def remove_nikud(self, text):
        """הסרת ניקוד"""
        nikud_chars = '\u05B0\u05B1\u05B2\u05B3\u05B4\u05B5\u05B6\u05B7\u05B8\u05B9\u05BA\u05BB\u05BC\u05BD\u05BE\u05BF\u05C0\u05C1\u05C2\u05C3\u05C4\u05C5\u05C6\u05C7'
        return ''.join(char for char in text if char not in nikud_chars)

    def generate_training_data(self):
        """יצירת נתוני אימון מובנים"""

        training_examples = []

        for sentence in self.training_sentences:
            labels = self.create_token_labels(
                sentence['without_nikud'],
                sentence['with_nikud']
            )

            if labels:
                training_examples.append({
                    'text': sentence['without_nikud'],
                    'labels': labels,
                    'context': sentence['context'],
                    'target': sentence['with_nikud']
                })

        return training_examples

# יצירת נתוני האימון
data_generator = TalmudTrainingDataGenerator()
training_data = data_generator.generate_training_data()

print(f"📊 נוצרו {len(training_data)} דוגמאות אימון")

# הצגת דוגמה
if training_data:
    example = training_data[0]
    print(f"\n📝 דוגמת אימון:")
    print(f"טקסט: {example['text']}")
    print(f"יעד: {example['target']}")
    print(f"הקשר: {example['context']}")
    print(f"תוויות: {example['labels'][:10]}...")  # 10 הראשונות


⚠️ אי-התאמה: 'רבנן סברי הכי ורבי יהודה סבר הכי' vs 'רבנן סבירי הכי ורבי יהודה סבר הכי'
⚠️ אי-התאמה: 'ואמרינן בגמרא דהלכה כרבי יהודה' vs 'ואמרינן בגמרא דהלכתא כרבי יהודה'
📊 נוצרו 8 דוגמאות אימון

📝 דוגמת אימון:
טקסט: רבי יהודה אמר דכתיב שלושה ימים
יעד: רַבִּי יְהוּדָה אָמַר דִּכְתִיב שְׁלוֹשָׁה יָמִים
הקשר: הלכה
תוויות: ['ַ', 'ִּ', 'NO_NIKUD', 'SPACE', 'ְ', 'NO_NIKUD', 'ּ', 'ָ', 'NO_NIKUD', 'SPACE']...
